In [1]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np

In [2]:
train_set = pd.read_csv("./datasets/mnist/train.csv")
test_set = pd.read_csv("./datasets/mnist/test.csv")

In [3]:
y_train = train_set["label"]
X_train = train_set.drop("label", axis=1)
X_test = test_set
x_split_len = int(len(X_train) * 0.30)

X_train, X_valid = X_train[x_split_len:], X_train[:x_split_len]
y_train, y_valid = y_train[x_split_len:], y_train[:x_split_len]

In [4]:
X_train = tf.reshape(X_train, [-1, 28, 28, 1])
X_valid = tf.reshape(X_valid, [-1, 28, 28, 1])
X_test = tf.reshape(X_test, [-1, 28, 28, 1])

In [5]:
keras.backend.clear_session()

model = keras.models.Sequential([
    keras.layers.Conv2D(8, kernel_size=3, padding="same", activation="relu", input_shape=[28, 28, 1]),
    keras.layers.Conv2D(16, kernel_size=3, padding="same", activation="relu"),
    keras.layers.MaxPooling2D(2),
    keras.layers.Conv2D(64, kernel_size=5, padding="same", activation="relu"),
    keras.layers.MaxPooling2D(),
    keras.layers.Flatten(),
    keras.layers.Dropout(0.25),
    keras.layers.Dense(64, activation="relu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(10, activation="softmax"),
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 8)         80        
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 16)        1168      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 16)       0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 14, 14, 64)        25664     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 7, 7, 64)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 3136)              0

In [6]:
model.compile(loss="sparse_categorical_crossentropy", optimizer=keras.optimizers.Nadam(), metrics=["accuracy"])

cb_earlystopping = keras.callbacks.EarlyStopping(patience=10)
cb_checkpoint = keras.callbacks.ModelCheckpoint("./models/my_mnist.h5", save_best_only=True)

history = model.fit(X_train, y_train, epochs=100, validation_data=(X_valid, y_valid), callbacks=[cb_earlystopping, cb_checkpoint])

Epoch 1/100
919/919 [==============================] - 15s 14ms/step - loss: 0.8401 - accuracy: 0.7334 - val_loss: 0.1070 - val_accuracy: 0.9683
Epoch 2/100
919/919 [==============================] - 12s 14ms/step - loss: 0.2424 - accuracy: 0.9267 - val_loss: 0.0886 - val_accuracy: 0.9754
Epoch 3/100
919/919 [==============================] - 13s 14ms/step - loss: 0.1678 - accuracy: 0.9494 - val_loss: 0.0558 - val_accuracy: 0.9849
Epoch 4/100
919/919 [==============================] - 13s 14ms/step - loss: 0.1339 - accuracy: 0.9607 - val_loss: 0.0518 - val_accuracy: 0.9840
Epoch 5/100
919/919 [==============================] - 12s 13ms/step - loss: 0.1170 - accuracy: 0.9643 - val_loss: 0.0460 - val_accuracy: 0.9866
Epoch 6/100
919/919 [==============================] - 12s 13ms/step - loss: 0.0984 - accuracy: 0.9713 - val_loss: 0.0453 - val_accuracy: 0.9879
Epoch 7/100
919/919 [==============================] - 12s 13ms/step - loss: 0.0910 - accuracy: 0.9733 - val_loss: 0.0427 - val_ac

In [7]:
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, 1)

pd_submit = pd.read_csv('./datasets/mnist/sample_submission.csv')
pd_submit['Label'] = y_pred
pd_submit.to_csv('./datasets/mnist/mnist_result.csv', header=True, index=False)